In [67]:
import numpy as np 
import matplotlib.pyplot as plt
import os
import cv2 as cv 
from os.path import join
import pydicom as dicom
from skimage import exposure
from skimage import io, morphology
from skimage import filters
import progressbar 
%matplotlib qt

In [95]:
#path of the images
path = r'D:\Dalhousie\datasets'
#folder = r'05 2023.07.30 NoFiducials\cropped'
#folder = r'06 2023.07.30 WithFiducials\cropped'
folder = r'Extracorporeal\Plane' #using images image_3139088460310 and image_3172566532381

saving_folder = r'06 2023.07.30 WithFiducials\LogGabor'

in_path=join(path,folder)
entries = os.listdir(in_path)
image_path = join(in_path,entries[0]) #Selected image
savedimage_path = join(path,saving_folder)



In [89]:
img=cv.imread(image_path,0)
plt.figure()
plt.imshow(img,cmap='gray')
#cv.imshow('Imagen original',img)


mask = np.zeros(img.shape,dtype=np.uint8)#


mask[80:641,359:912]=255 #In case of using Extracorporeal Planes images, and want to remove the US text and measurements 
img[mask==0]=0


#img = cv.fastNlMeansDenoising(img, h=12, templateWindowSize = 7,	searchWindowSize = 25)
cv.imshow('Imagen original',img)
cv.waitKey(0)
#img.shape

#plt.figure()
#plt.imshow(img,cmap='gray')

-1

In [90]:
def gammaCorrection(src,invGamma):
    table = [((i / 255) ** invGamma) * 255 for i in range(256)]
    table = np.array(table, np.uint8)
    return cv.LUT(src, table)

In [91]:
#Function to create a frequency grid to create the filter.
def Grid2D(imag_shape):
    ysize,xsize = imag_shape #The grid needs to be the same size as the image
    ymid = ysize // 2
    xmid = xsize // 2
    
    if ysize % 2 == 0: #Condition to led with images with odd or even size.
        ymax = ymid - 1
    else:
        ymax = ymid

    if xsize % 2 == 0:
        xmax = xmid - 1
    else:
        xmax = xmid
    
    X,Y= np.meshgrid(np.arange(-xmid, xmax + 1),np.arange(-ymid, ymax + 1)) #Spatial domain of the grid
    X = np.fft.ifftshift(X)/xsize #Frecuency domain of the grid (also normalized)
    Y = np.fft.ifftshift(Y)/ysize
    
    return X,Y

# One scale approach

In [101]:
def RiezFilter(img_shape): 
    u,v = Grid2D(img_shape)
    w = np.sqrt(u**2+v**2)
    w[0,0]=0.0001
    
    H1 = (1j * u)/ w
    H2 = (1j * v)/ w
    
    return H1,H2

def LogGaborFilter(img_shape,scaling_factor,wavelenght,k_w0):
    
    #Create a Frequency Grid
    X,Y = Grid2D(img_shape)
    
    #Determine the spatial regions to use 
    w = np.sqrt(Y**2+X**2)
    w[0,0]=0.0001 #Avoids log(0)
    phi = np.arctan2(Y,X)
    
    #User defined parameter
    m = scaling_factor
    
    #for i in range(len(wavelenght)):
        
    lamb = wavelenght #[i]
    w_0 = 2/(lamb*3)

    GabFilter = np.exp( -((np.log(w/w_0))**2 / (2*(np.log(k_w0))**2)))#+((phi-phi_0)**2/(2*sig_phi**2)) 
    GabFilter[0,0] = 0

    if (img_shape[0]%2 == 0):
        GabFilter[img_shape[0]//2,:]=0

    if (img_shape[1]%2 == 0):
        GabFilter[:,img_shape[1]//2]=0

    #FilterBank[i,:,:] = GabFilter
    return GabFilter

def MonogenicSignal(image,LogGaborFilt,H1,H2):
    
    F = np.fft.fft2(image)
    Ffilt = np.multiply(F,LogGaborFilt)
    
    F_g = np.fft.ifft2(Ffilt)
    
    F_gi = np.real(F_g)
    fm1 = np.real(F_g)
    
    F_modd_1 = np.multiply(Ffilt,H1) 
    F_modd_1 = np.fft.ifft2(F_modd_1)
    fm2 = np.real(F_modd_1)
    #print(np.min(np.imag(F_modd_1)))
    
    F_modd_2 = np.multiply(Ffilt,H2) 
    F_modd_2 = np.fft.ifft2(F_modd_2)
    fm3 = np.real(F_modd_2)
    
    return fm1,fm2,fm3


def FeatureSymetry(m1,m2,m3):
    epsilon = 0.01
    
    odd = np.sqrt(m2**2 + m3**2)
    even = np.abs(m1)
    
    
    Amp = np.sqrt(even**2+odd**2)
    T = np.exp(np.log(np.mean(np.sqrt(odd**2+even**2))))
    
    denominator = np.sqrt(even**2+odd**2)+epsilon
    
    FS_num = np.clip((even - odd - T), 0,None)
    FA_num = np.clip((odd - even - T), 0,None)
    
    FS = FS_num/denominator
    FA = FA_num/denominator
    
    #FS = np.mean(FS, axis=0)
    #FA = np.mean(FA, axis=0)
    
    return FS,FA

H1,H2 = RiezFilter(img.shape)
wl = 36

lG = LogGaborFilter(img.shape,1.5,wl,0.55)

m1,m2,m3 = MonogenicSignal(img,lG,H1,H2)

#LocalAmp = np.sqrt(m1**2+m2**2+m3**2)
#LocalPhase = np.arctan2((np.sqrt(m2**2+m3**2)),m1)

FS,FA = FeatureSymetry(m1,m2,m3)

FA = (FA-FA.min())*(255/(FA.max()-FA.min()))
FA = FA.astype(np.uint8)
FS = (FS-FS.min())*(255/(FS.max()-FS.min()))
FS = FS.astype(np.uint8)
Amp = (Amp-Amp.min())*(255/(Amp.max()-Amp.min()))
Amp = Amp.astype(np.uint8)


cv.imshow('',np.c_[img,FS,FA])
cv.waitKey(0)


-1

In [108]:
maximumvalue = len(entries)


bar = progressbar.ProgressBar(maxval=maximumvalue-1).start()

for ith in range (0,maximumvalue):
    
    in_image_path = join(in_path,entries[ith])
    out_image_path = join(savedimage_path,entries[ith])

    img = cv.imread(in_image_path,0)
    
    mask[80:641,359:912]=255 #In case of using Extracorporeal Planes images, and want to remove the US text and measurements 
    img[mask==0]=0
    
    H1,H2 = RiezFilter(img.shape)
    wl = 60

    lG = LogGaborFilter(img.shape,1.5,wl,0.25)

    m1,m2,m3 = MonogenicSignal(img,lG,H1,H2)


    FS,FA = FeatureSymetry(m1,m2,m3)

    FA = (FA-FA.min())*(255/(FA.max()-FA.min()))
    FA = FA.astype(np.uint8)
    FS = (FS-FS.min())*(255/(FS.max()-FS.min()))
    FS = FS.astype(np.uint8)


    cv.imshow('',np.c_[img,FA,FS])
    cv.waitKey()
    #cv.imwrite(out_image_path,FS)
    bar.update(ith)

100% (1 of 1) |##########################| Elapsed Time: 0:00:04 ETA:  00:00:00

# Multi-scale appoach

In [109]:
def LogGaborFilter_MS(img_shape,scaling_factor,wavelenght,k_w0):
    
    #Create a Frequency Grid
    X,Y = Grid2D(img_shape)
    
    FiltBank = np.zeros((len(wavelenght),img_shape[0],img_shape[1]))
    
    #Determine the spatial regions to use 
    w = np.sqrt(Y**2+X**2)
    w[0,0]=0.0001 #Avoids log(0)
    phi = np.arctan2(Y,X)
    
    #User defined parameter
    m = scaling_factor
    
    for i in range(len(wavelenght)):
        
        lamb = wavelenght[i]
        w_0 = 2/(lamb*3)

        GabFilter = np.exp( -((np.log(w/w_0))**2 / (2*(np.log(k_w0))**2)))#+((phi-phi_0)**2/(2*sig_phi**2))) 
        GabFilter[0,0] = 0

        if (img_shape[0]%2 == 0):
            GabFilter[img_shape[0]//2,:]=0

        if (img_shape[1]%2 == 0):
            GabFilter[:,img_shape[1]//2]=0

        FiltBank[i,:,:] = GabFilter
        
    return FiltBank



def FeatureSymetry_MS(m1,m2,m3):
    epsilon = 0.01
    
    odd = np.sqrt(m2**2 + m3**2)
    even = np.abs(m1)
    
    T = np.exp(np.log(np.mean(np.sqrt(odd**2+even**2))))
    
    denominator = np.sqrt(even**2+odd**2)+epsilon
    
    FS_num = np.clip((even - odd - T), 0,None)
    FA_num = np.clip((odd - even - T), 0,None)
    
    FS = FS_num/denominator
    FA = FA_num/denominator
    
    FS = np.sum(FS, axis=0)
    FA = np.sum(FA, axis=0)
    
    
    return FS,FA

def PhaseConcruency(m1,m2,m3):
    
    epsilon = 0.01
    
    odd = np.sqrt(m2**2 + m3**2)
    even = np.abs(m1)
    
    print(odd.shape)
    
    Fx = np.sum(even,axis= 0)
    Hx = np.sum(odd,axis=0)
    
    Energy = np.sqrt(Fx**2+Hx**2)
    print(Fx.shape,Hx.shape)
    
    Amp = np.sqrt(odd**2+even**2)
    
    A0 = np.exp(np.log(np.mean(Amp[0,:,:])))
    T = 2.5*A0*((1-(1/1.5)**6)/(1-(1/1.5))) 
    
    Amp = np.sum(Amp,axis=0)
    
    s = (1/6)*(Amp/(epsilon+np.max(Amp)))
    W = 1/(1+np.exp(10*(0.4-(s))))
    
    PC = np.maximum((W*(Energy-T)),0)/(Amp+epsilon)
    
    return PC

print(img.shape)

def HystThresh(image):
    low = 0.3
    high = 0.5

    lowt = (image > low).astype(int)
    hight = (image > high).astype(int)
    hyst = filters.apply_hysteresis_threshold(image, low, high)
    print(high+hyst)

mask = np.zeros(img.shape,dtype=np.uint8)
mask[80:641,359:912]=255
img[mask==0]=0

H1,H2 = RiezFilter(img.shape)

wl = np.array([32,34,36])
print(wl)

lG = LogGaborFilter_MS(img.shape,1.5,wl,0.15)

m1,m2,m3 = MonogenicSignal(img,lG,H1,H2)


FS_m,FA_m = FeatureSymetry_MS(m1,m2,m3)

FA_m = (FA_m-FA_m.min())*(255/(FA_m.max()-FA_m.min()))
FA_m = FA_m.astype(np.uint8)
FS_m = (FS_m-FS_m.min())*(255/(FS_m.max()-FS_m.min()))
FS_m = FS_m.astype(np.uint8)


cv.imshow('',np.c_[img,FA_m,FS_m])
cv.waitKey(0)


(720, 1280)
[32 34 36]


-1

In [112]:
maximumvalue = len(entries)


bar = progressbar.ProgressBar(maxval=maximumvalue-1).start()

for ith in range (0,maximumvalue):
    
    in_image_path = join(in_path,entries[ith])
    out_image_path = join(savedimage_path,entries[ith])
    
    img = cv.imread(in_image_path,0)
    
    mask = np.zeros(img.shape,dtype=np.uint8)
    mask[80:641,359:912]=255
    img[mask==0]=0
    
    
    H1,H2 = RiezFilter(img.shape)
    wl = np.array([30,32,34,36])

    lG = LogGaborFilter_MS(img.shape,1.5,wl,0.15)

    m1,m2,m3 = MonogenicSignal(img,lG,H1,H2)


    FS_m,FA_m = FeatureSymetry_MS(m1,m2,m3)

    FA_m = (FA_m-FA_m.min())*(255/(FA_m.max()-FA_m.min()))
    FA_m = FA_m.astype(np.uint8)
    FS_m = (FS_m-FS_m.min())*(255/(FS_m.max()-FS_m.min()))
    FS_m = FS_m.astype(np.uint8)

    
    
    cv.imshow('',np.c_[img,FS_m,FA_m])

    cv.imwrite(out_image_path,np.c_[img,FS_m])
    cv.waitKey(0)
    bar.update(ith)

100% (1 of 1) |##########################| Elapsed Time: 0:00:06 ETA:  00:00:00

# MRI images

In [113]:


maximumvalue = 160


bar = progressbar.ProgressBar(maxval=maximumvalue).start()

for ith in range (3,maximumvalue):
    I = dicom.dcmread('C:/Users/eddea/Desktop/Tesis Maestría/Codes/Original/'+str(ith)+'.dcm')
    I=I.pixel_array
    I=exposure.equalize_adapthist(I)
    img = I
    
    H1,H2 = RiezFilter(img.shape)
    wl = np.array([2,3,5])

    lG = LogGaborFilter_MS(img.shape,1.5,wl,0.15)

    m1,m2,m3 = MonogenicSignal(img,lG,H1,H2)


    FS_m,FA_m = FeatureSymetry_MS(m1,m2,m3)

    FA_m = (FA_m-FA_m.min())*(255/(FA_m.max()-FA_m.min()))
    FA_m = FA_m.astype(np.uint8)
    FS_m = (FS_m-FS_m.min())*(255/(FS_m.max()-FS_m.min()))
    FS_m = FS_m.astype(np.uint8)
    
    
    cv.imshow('',np.c_[img,FS_m,FA_m])

    cv.waitKey(30)
    bar.update(ith)

 99% (159 of 160) |##################### | Elapsed Time: 0:00:18 ETA:   0:00:00

# Video Processing

In [116]:
cap = cv.VideoCapture('D:/Dalhousie/datasets/Extracorporeal/Videos/image_12936930724906.mp4')

def processvideo(img):
    H1,H2 = RiezFilter(img.shape)
    wl = np.array([30,32,34,36])

    lG = LogGaborFilter_MS(img.shape,1.5,wl,0.15)

    m1,m2,m3 = MonogenicSignal(img,lG,H1,H2)


    FS_m,FA_m = FeatureSymetry_MS(m1,m2,m3)

    FA_m = (FA_m-FA_m.min())*(255/(FA_m.max()-FA_m.min()))
    FA_m = FA_m.astype(np.uint8)
    FS_m = (FS_m-FS_m.min())*(255/(FS_m.max()-FS_m.min()))
    FS_m = FS_m.astype(np.uint8)
    
    Cleaned = np.zeros((FS_m.shape[0],FS_m.shape[1]),np.uint8)
    
    Cleaned_Upper = MaskCorrection(FS_m[:FS_m.shape[0]//2,:],'u')
    Cleaned_Lower = MaskCorrection(FS_m[FS_m.shape[0]//2:,:],'l')
    Cleaned[:FS_m.shape[0]//2,:] = Cleaned_Upper
    Cleaned[FS_m.shape[0]//2:,:] = Cleaned_Lower
    
    
    FS_M = FS_m.copy()
    FS_M[Cleaned[:,:] == 255] = 255
    
    
    return np.c_[img,FS_m,FS_M]


if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # Our operations on the frame come here
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    processed = processvideo(gray)

    # Display the resulting frame
    shape = frame.shape
    
    cv.imshow('frame', processed)

    if cv.waitKey(1) == ord('q'):
        break

cap.release()

# Post-Processing Stage

In [115]:
def MaskCorrection(img,ul):
    I = img.copy()
    
    kernel = cv.getStructuringElement(cv.MORPH_RECT,(5,5))
    Cleaned = cv.morphologyEx(I, cv.MORPH_GRADIENT, kernel)
    Arr = I>0
    
    if ul == 'u':
        I[I<50]=0
        I[I>=50]=255
        Cleaned = morphology.remove_small_objects(Arr, min_size=1000)
        Cleaned = morphology.skeletonize(Cleaned)
        Cleaned= Cleaned.astype(np.uint8)*255
        
    if ul == 'l':
        I = gammaCorrection(I,.15)
        I[I<10]=0
        I[I>=10]=255
        Cleaned = morphology.remove_small_objects(Arr, min_size=100)
        Cleaned = morphology.skeletonize(Cleaned)
        labeled,num_objects = morphology.label(Cleaned, connectivity=2, return_num = True)
        Cleaned= Cleaned.astype(np.uint8)*255
        Cleaned = cv.dilate(Cleaned,kernel,iterations=5)
        
        areas = np.zeros(num_objects)
        for objects in range (num_objects):
            areas[objects] = len(labeled[labeled==objects])
            
        
        arguments = areas.argsort()[-4:][::-1]
        arguments = arguments[1:]
        
        if arguments.shape[0] > 0:
            Cleaned[labeled != arguments[0]] = 0
            Cleaned = cv.dilate(Cleaned,kernel,2)
    
    return Cleaned